# Overview
Spark SQL is a Spark module for structured data processing. Unlike the basic Spark RDD API, the interfaces provided by Spark SQL provide Spark with more information about the structure of both the data and the computation being performed. Internally, Spark SQL uses this extra information to perform extra optimizations. There are several ways to interact with Spark SQL including SQL, the DataFrames API and the Datasets API. When computing a result the same execution engine is used, independent of which API/language you are using to express the computation. This unification means that developers can easily switch back and forth between the various APIs based on which provides the most natural way to express a given transformation.

All of the examples on this page use sample data included in the Spark distribution and can be run in the spark-shell, pyspark shell, or sparkR shell.

## SQL
One use of Spark SQL is to execute SQL queries written using either a basic SQL syntax or HiveQL. Spark SQL can also be used to read data from an existing Hive installation. For more on how to configure this feature, please refer to the Hive Tables section. When running SQL from within another programming language the results will be returned as a DataFrame. You can also interact with the SQL interface using the command-line or over JDBC/ODBC.

## DataFrames
A DataFrame is a distributed collection of data organized into named columns. It is conceptually equivalent to a table in a relational database or a data frame in R/Python, but with richer optimizations under the hood. DataFrames can be constructed from a wide array of sources such as: structured data files, tables in Hive, external databases, or existing RDDs.

The DataFrame API is available in Scala, Java, Python, and R.

## Datasets
A Dataset is a new experimental interface added in Spark 1.6 that tries to provide the benefits of RDDs (strong typing, ability to use powerful lambda functions) with the benefits of Spark SQL’s optimized execution engine. A Dataset can be constructed from JVM objects and then manipulated using functional transformations (map, flatMap, filter, etc.).

The unified Dataset API can be used both in Scala and Java. Python does not yet have support for the Dataset API, but due to its dynamic nature many of the benefits are already available (i.e. you can access the field of a row by name naturally row.columnName). Full python support will be added in a future release.

In [5]:
// Creating SparkContext
// val sc: SparkContext // An existing SparkContext.
// val sqlContext = new org.apache.spark.sql.SQLContext(sc)
println("SparkContext")
for ( (k, v) <- sc.getConf.getAll) {
    println(s"$k: $v")
}
println("\nSQLContext)
for ( (k, v) <- sqlContext.getAllConfs) {
    println(s"$k: $v")
}

yarn.resourcemanager.amlauncher.thread-count ---> 50
hive.server2.thrift.http.cookie.is.httponly ---> true
dfs.namenode.resource.check.interval ---> 5000
yarn.sharedcache.enabled ---> false
fs.s3a.connection.maximum ---> 15
mapreduce.jobtracker.jobhistory.task.numberprogresssplits ---> 12
mapreduce.tasktracker.healthchecker.script.timeout ---> 600000
fs.s3a.impl ---> org.apache.hadoop.fs.s3a.S3AFileSystem
yarn.app.mapreduce.am.scheduler.heartbeat.interval-ms ---> 1000
mapreduce.input.fileinputformat.input.dir.recursive ---> false
hive.orc.compute.splits.num.threads ---> 10
hadoop.proxyuser.root.hosts ---> *
hive.auto.convert.sortmerge.join.to.mapjoin ---> false
hadoop.http.authentication.kerberos.principal ---> HTTP/_HOST@LOCALHOST
hive.vectorized.execution.reduce.groupby.enabled ---> true
hive.exec.perf.logger ---> org.apache.hadoop.hive.ql.log.PerfLogger
hive.tez.max.partition.factor ---> 2.0
mapreduce.framework.name ---> yarn
yarn.sharedcache.uploader.server.thread-count ---> 50
hiv

##  Yelp Data
Let's use out Yelp data
<pre>
$ hdfs dfs -ls /user/pmolnar/yelp/data
Found 5 items
drwxr-xr-x   - pmolnar hadoop          0 2017-01-15 12:12 /user/pmolnar/yelp/data/business
drwxr-xr-x   - pmolnar hadoop          0 2017-01-15 12:14 /user/pmolnar/yelp/data/checkin
dr-xr-xr-x   - pmolnar hadoop          0 2017-01-15 13:02 /user/pmolnar/yelp/data/review
drwxr-xr-x   - pmolnar hadoop          0 2017-01-15 12:23 /user/pmolnar/yelp/data/tip
drwxr-xr-x   - pmolnar hadoop          0 2017-01-15 12:24 /user/pmolnar/yelp/data/user
</pre>

In [8]:
val DATADIR="/user/pmolnar/yelp/data"

In [9]:
val user_df = sqlContext.read.json(s"$DATADIR/user/")

In [10]:
user_df.printSchema

root
 |-- average_stars: double (nullable = true)
 |-- compliments: struct (nullable = true)
 |    |-- cool: long (nullable = true)
 |    |-- cute: long (nullable = true)
 |    |-- funny: long (nullable = true)
 |    |-- hot: long (nullable = true)
 |    |-- list: long (nullable = true)
 |    |-- more: long (nullable = true)
 |    |-- note: long (nullable = true)
 |    |-- photos: long (nullable = true)
 |    |-- plain: long (nullable = true)
 |    |-- profile: long (nullable = true)
 |    |-- writer: long (nullable = true)
 |-- elite: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- fans: long (nullable = true)
 |-- friends: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- name: string (nullable = true)
 |-- review_count: long (nullable = true)
 |-- type: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- votes: struct (nullable = true)
 |    |-- cool: long (nullable = true)
 |    |-- funny: long (nullable = true)

In [11]:
val business_df = sqlContext.read.json(s"$DATADIR/business/")

In [12]:
business_df.printSchema

root
 |-- attributes: struct (nullable = true)
 |    |-- Accepts Credit Cards: boolean (nullable = true)
 |    |-- Accepts Insurance: boolean (nullable = true)
 |    |-- Ages Allowed: string (nullable = true)
 |    |-- Alcohol: string (nullable = true)
 |    |-- Ambience: struct (nullable = true)
 |    |    |-- casual: boolean (nullable = true)
 |    |    |-- classy: boolean (nullable = true)
 |    |    |-- divey: boolean (nullable = true)
 |    |    |-- hipster: boolean (nullable = true)
 |    |    |-- intimate: boolean (nullable = true)
 |    |    |-- romantic: boolean (nullable = true)
 |    |    |-- touristy: boolean (nullable = true)
 |    |    |-- trendy: boolean (nullable = true)
 |    |    |-- upscale: boolean (nullable = true)
 |    |-- Attire: string (nullable = true)
 |    |-- BYOB: boolean (nullable = true)
 |    |-- BYOB/Corkage: string (nullable = true)
 |    |-- By Appointment Only: boolean (nullable = true)
 |    |-- Caters: boolean (nullable = true)
 |    |-- Coat Chec

In [13]:
val review_df = sqlContext.read.json(s"$DATADIR/review/")

In [14]:
review_df.printSchema

root
 |-- business_id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- stars: long (nullable = true)
 |-- text: string (nullable = true)
 |-- type: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- votes: struct (nullable = true)
 |    |-- cool: long (nullable = true)
 |    |-- funny: long (nullable = true)
 |    |-- useful: long (nullable = true)



In [15]:
val checkin_df = sqlContext.read.json(s"$DATADIR/checkin/")
checkin_df.printSchema

root
 |-- business_id: string (nullable = true)
 |-- checkin_info: struct (nullable = true)
 |    |-- 0-0: long (nullable = true)
 |    |-- 0-1: long (nullable = true)
 |    |-- 0-2: long (nullable = true)
 |    |-- 0-3: long (nullable = true)
 |    |-- 0-4: long (nullable = true)
 |    |-- 0-5: long (nullable = true)
 |    |-- 0-6: long (nullable = true)
 |    |-- 1-0: long (nullable = true)
 |    |-- 1-1: long (nullable = true)
 |    |-- 1-2: long (nullable = true)
 |    |-- 1-3: long (nullable = true)
 |    |-- 1-4: long (nullable = true)
 |    |-- 1-5: long (nullable = true)
 |    |-- 1-6: long (nullable = true)
 |    |-- 10-0: long (nullable = true)
 |    |-- 10-1: long (nullable = true)
 |    |-- 10-2: long (nullable = true)
 |    |-- 10-3: long (nullable = true)
 |    |-- 10-4: long (nullable = true)
 |    |-- 10-5: long (nullable = true)
 |    |-- 10-6: long (nullable = true)
 |    |-- 11-0: long (nullable = true)
 |    |-- 11-1: long (nullable = true)
 |    |-- 11-2: long (nul

In [17]:
val tip_df = sqlContext.read.json(s"$DATADIR/tip/")
tip_df.printSchema

root
 |-- business_id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- likes: long (nullable = true)
 |-- text: string (nullable = true)
 |-- type: string (nullable = true)
 |-- user_id: string (nullable = true)



In [21]:
tip_df.groupBy("likes").count().show()

+-----+-----+
|likes|count|
+-----+-----+
|    0|64201|
|    1|  630|
|    2|   50|
|    3|    8|
|    5|    1|
+-----+-----+



In [37]:
tip_df.filter(tip_df.col("likes")===5).select("date", "text").collect

Array([2014-12-17,This drive-thru is sketchy and narrow. How do people do this while under the influence!?])

In [69]:
tip_df.filter("likes >= 3").
    join(business_df, tip_df("business_id") === business_df("business_id")).
    select("name", "stars", "likes", "text", "state").
    collect

Array([Phoenix Sky Harbor International Airport - PHX,3.5,3,PArking was a breeze, sending the fam back to the bay!,AZ], [In-N-Out Burger,4.0,3,Get a 3x3 it's 3 patties and 3 slices of cheese plus the fixings for $4,AZ], [Filiberto's Mexican Food,2.5,5,This drive-thru is sketchy and narrow. How do people do this while under the influence!?,AZ], [Little Dumpling,3.5,3,This is the lunch special menu,NV], [Auntie Anne's,4.0,3,Got Yu-go his birthday pretzel and mini hotdog pretzels too !,NV], [Fatburger,3.0,3,Koroppi is with the crispy chicken burger.  Did you know that Magic Johnson once owned this burger chain.,NV], [Sonoma Cellar,4.0,3,Carrot cake with French pressed coffee 5/5,NV], [Sonoma Cellar,4.0,3,Four layer rich dark chocolate cake and frenc...

In [77]:
tip_df.
    join(business_df, tip_df("business_id") === business_df("business_id")).
    where("likes >= 3 and state='AZ'").
    select("name", "stars", "likes", "text").
    collect

Array([Phoenix Sky Harbor International Airport - PHX,3.5,3,PArking was a breeze, sending the fam back to the bay!], [In-N-Out Burger,4.0,3,Get a 3x3 it's 3 patties and 3 slices of cheese plus the fixings for $4], [Filiberto's Mexican Food,2.5,5,This drive-thru is sketchy and narrow. How do people do this while under the influence!?])

In [ ]:

// Show the content of the DataFrame
df.show()
// age  name
// null Michael
// 30   Andy
// 19   Justin

// Print the schema in a tree format
df.printSchema()
// root
// |-- age: long (nullable = true)
// |-- name: string (nullable = true)

// Select only the "name" column
df.select("name").show()
// name
// Michael
// Andy
// Justin

// Select everybody, but increment the age by 1
df.select(df("name"), df("age") + 1).show()
// name    (age + 1)
// Michael null
// Andy    31
// Justin  20

// Select people older than 21
df.filter(df("age") > 21).show()
// age name
// 30  Andy

// Count people by age
df.groupBy("age").count().show()
// age  count
// null 1
// 19   1
// 30   1

Flat map reviews text:

In [105]:
review_df.take(4).
    map(s => s(4).toString.replaceAll("[.;,!]", "").toLowerCase).
    flatMap(s => s.split(' '))

Array(mr, hoagie, is, an, institution, walking, in, it, does, seem, like, a, throwback, to, 30, years, ago, old, fashioned, menu, board, booths, out, of, the, 70s, and, a, large, selection, of, food, their, speciality, is, the, italian, hoagie, and, it, is, voted, the, best, in, the, area, year, after, year, i, usually, order, the, burger, while, the, patties, are, obviously, cooked, from, frozen, all, of, the, other, ingredients, are, very, fresh, overall, its, a, good, alternative, to, subway, which, is, down, the, road, excellent, food, superb, customer, service, i, miss, the, mario, machines, they, used, to, have, but, it's, still, a, great, place, steeped, in, tradition, yes, this, place, is, a, little, out, dated, and, not, opened, on, the, weekend, but, o...

In [ ]:
"sdf".toString.replace